In [1]:
from trimesh import *
from swf import *
from optimal import *
from utils import *
import numpy as np
import soundfile as sf
from scipy import spatial

import fileinput
import sys

def random_three_vector():
    """
    Generates a random 3D unit vector (direction) with a uniform spherical distribution
    Algo from http://stackoverflow.com/questions/5408276/python-uniform-spherical-distribution
    :return:
    """
    phi = np.random.uniform(0,np.pi*2)
    costheta = np.random.uniform(-1,1)

    theta = np.arccos( costheta )
    x = np.sin( theta) * np.cos( phi )
    y = np.sin( theta) * np.sin( phi )
    z = np.cos( theta )
    return (x,y,z)

In [2]:
layout704=np.array([[1,np.pi/6,np.pi/2],[1,-np.pi/6,np.pi/2],[1,0,np.pi/2],[1,np.pi/2,np.pi/2],[1,-np.pi/2,np.pi/2],[1,3*np.pi/4,np.pi/2],[1,-3*np.pi/4,np.pi/2],[1,np.pi/4,np.pi/4],[1,-np.pi/4,np.pi/4],[1,3*np.pi/4,np.pi/4],[1,-3*np.pi/4,np.pi/4]])
vertices704 = np.apply_along_axis(lambda x: toCartesian(x),1,layout704)
faces704 = np.array([[6,4,10],[10,4,8],[8,4,1],[8,1,2],[8,7,2],[7,2,0],[7,0,3],[7,3,9],[9,3,5],[10,7,9],[10,8,7],[10,6,5],[10,9,5]])

model = OptimalSWF(vertices704,faces704).model

In [3]:
data, samplerate = sf.read('bossa.wav')
tree = spatial.KDTree(model.meshes[-1].vertices)

In [4]:
#Compute encoding for immobile point source 
fine = np.zeros((model.meshes[-1].vertices.shape[0],data.shape[0]))
loc = tree.query([random_three_vector()])[1][0]
onehot = np.zeros((model.meshes[-1].vertices.shape[0],1))
onehot[:11] = -1
onehot[loc]=1
fine[loc] = data
coarse = model.encode(fine)
coarse = np.insert(coarse, 3, 0, axis=0) #add empty subwoofer channel for reproduction in the studio

#weights3D(model.meshes[-1],onehot,'point source location') #graph it

sf.write('pointSource.wav',coarse.T,samplerate)

In [5]:
#Compute a Horizontal Panning
fine = np.zeros((model.meshes[-1].vertices.shape[0],data.shape[0]))
theta = np.linspace(0,2*np.pi,data.shape[0])
x = np.cos(theta)
y = np.sin(theta)

horizontalPanning = np.vstack((x,y,np.zeros(theta.shape[0]))).T

dist, ind = tree.query(horizontalPanning, k=3) 

vxtot = model.meshes[-1].vertices[ind]
vxtot[:,:,2] = 1
Atot = np.abs(np.linalg.det(vxtot))/2
panningVerts = horizontalPanning.reshape((-1,1,3))
panningVerts[:,:,2] = 1

A0 = np.abs(np.linalg.det(np.hstack((panningVerts,vxtot[:,1:,:]))))/2
A1 = np.abs(np.linalg.det(np.hstack((vxtot[:,0,:].reshape(-1,1,3),panningVerts,vxtot[:,2,:].reshape(-1,1,3)))))/2
A2 = np.abs(np.linalg.det(np.hstack((vxtot[:,:2,:],panningVerts))))/2

interpolation = np.vstack((A0/Atot,A1/Atot,A2/Atot)).T
for i in range(data.shape[0]):
    fine[ind[i],i] = data[i]*interpolation[i]
coarse = model.encode(fine)
coarse = np.insert(coarse, 3, 0, axis=0) #add empty subwoofer channel for reproduction in the studio

sf.write('horizontalPan.wav',coarse.T,samplerate)

In [6]:
#save encoder and vertex arrays for use in Max MSP
encoder = model.phi2s[0]
np.savetxt('encoder.txt',encoder,newline=';\n')
for line in fileinput.input('encoder.txt',inplace=True):
    sys.stdout.write('%d, %s'%(fileinput.filelineno(), line))
np.savetxt('vertices.txt',model.meshes[-1].vertices,newline=';\n')
for line in fileinput.input('vertices.txt',inplace=True):
    sys.stdout.write('%d, %s'%(fileinput.filelineno(), line))